In [9]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_community.document_loaders.csv_loader import CSVLoader
from langchain.chains.combine_documents import create_stuff_documents_chain
from dotenv import load_dotenv
import pandas as pd

In [10]:
#Carrega as variaveis de ambiente
load_dotenv()

True

In [11]:
file_path = r'vendas.csv'

In [12]:
loader = CSVLoader(file_path,
                   csv_args={
                       'delimiter': ',',
                       'fieldnames': ['mes',
                                      'ano',
                                      'medicamentos',
                                      'qtd_vendas',
                                      'trimestre',
                                      'semestre',
                                      'data']
                        }
                    )
data = loader.load()

In [13]:
llm = ChatOpenAI(
    model = "gpt-4o",
    temperature=0.3,
    max_tokens=1000,
    verbose=True
)

In [14]:
prompt = ChatPromptTemplate.from_template(
    """
        Você é um assistente para um gerente de uma farmácia de manipulação, chamado FarmaBot, 
        você será responsável por responder as perguntas do seu gerente 
        da maneira mais clara e precisa possível.
        
        O seu trabalho é ler o arquivo csv passado para você e responder as perguntas do seu gerente.
        com base nos dados do arquivo csv.
        
        Context: {context}
        
        Question: {input}
    """
)

In [15]:
chain = create_stuff_documents_chain(
    llm = llm,
    prompt= prompt,
    output_parser= StrOutputParser()
)

In [16]:
response = chain.invoke({
    "input": "Qual foi o remédio mais vendido no ano de 2022?",
    "context": data  
})

response

'Para determinar o remédio mais vendido no ano de 2022, precisamos somar as quantidades de vendas de cada medicamento ao longo do ano. Aqui estão as vendas totais de cada medicamento em 2022:\n\n- **Paracetamol**: 5 + 24 + 28 + 49 + 45 + 30 + 37 = 218\n- **Ibuprofeno**: 30 + 7 + 14 + 23 + 9 + 36 + 37 = 156\n- **Amoxicilina**: 13 + 21 + 42 + 3 + 3 + 21 + 50 = 153\n- **Captopril**: 19 + 40 + 38 + 30 + 44 + 29 + 34 = 234\n- **Atenolol**: 37 + 17 + 2 + 23 + 4 + 29 + 50 = 162\n- **Metformina**: 47 + 11 + 21 + 49 + 13 + 41 + 35 = 217\n- **Simvastatina**: 9 + 35 + 3 + 2 + 40 + 34 + 19 = 142\n- **Omeprazol**: 49 + 8 + 26 + 27 + 27 + 15 + 31 = 183\n- **Aspirina**: 31 + 13 + 23 + 30 + 16 + 45 + 39 = 197\n- **Losartana**: 27 + 17 + 3 + 46 + 31 + 36 + 44 = 204\n\nO remédio mais vendido no ano de 2022 foi **Captopril** com um total de 234 unidades vendidas.'

## Usando Agent

In [18]:
from langchain_experimental.agents.agent_toolkits import create_csv_agent
from langchain_openai import ChatOpenAI

In [19]:
llm = ChatOpenAI(
    model = "gpt-4o",
    temperature=0.3,
    max_tokens=1000,
    verbose=True
)

In [21]:
agent = create_csv_agent(llm, file_path, verbose=True,allow_dangerous_code=True)

In [ ]:
question = """
        Você é um assistente para um gerente de uma farmácia de manipulação, chamado FarmaBot, 
        você será responsável por responder as perguntas do seu gerente 
        da maneira mais clara e precisa possível, sempre respondendo em PT-BR.
        
        O seu trabalho é ler o arquivo csv passado para você e responder as perguntas do seu gerente.
        com base nos dados do arquivo csv.
        
        Qual foi o remédio mais vendido no ano de 2023?
"""

In [ ]:
agent.invoke(question)



> Entering new AgentExecutor chain...
Thought: To find the most sold medicine in 2023, I need to filter the dataframe for the year 2023 and then find the medicine with the highest sales.
Action: python_repl_ast
Action Input: df[df['ano'] == 2023].groupby('medicamentos')['qtd_vendas'].sum().idxmax()AtenololEu agora sei a resposta final.

Final Answer: Atenolol

> Finished chain.


{'input': '\n        Você é um assistente para um gerente de uma farmácia de manipulação, chamado FarmaBot, \n        você será responsável por responder as perguntas do seu gerente \n        da maneira mais clara e precisa possível, sempre respondendo em PT-BR.\n        \n        O seu trabalho é ler o arquivo csv passado para você e responder as perguntas do seu gerente.\n        com base nos dados do arquivo csv.\n        \n        Qual foi o remédio mais vendido no ano de 2023?\n',
 'output': 'Atenolol'}

In [ ]:
df = pd.read_csv(file_path)

In [ ]:
region_vins_count = df.groupby(['ano', 'medicamentos'])['qtd_vendas'].sum().reset_index()

In [ ]:
region_vins_count[region_vins_count['ano']==2022].sort_values(by=['qtd_vendas'])

,ano,medicamentos,qtd_vendas
9,2022,Simvastatina,142
0,2022,Amoxicilina,153
4,2022,Ibuprofeno,156
2,2022,Atenolol,162
7,2022,Omeprazol,183
1,2022,Aspirina,197
5,2022,Losartana,204
6,2022,Metformina,217
8,2022,Paracetamol,218
3,2022,Captopril,234
